# 1. Attribute selection in classification problems

### Curse of dimensionality

**Question** What is "curse of dimensionality"? What does it mean in practice?

**Question** How can we reduce the number of attributes in classification problems?


Download file seizure.csv containing information about the recording of brain activity of people (description of the dataset: https://archive.ics.uci.edu/dataset/388/epileptic+seizure+recognition).
Each row contains 178 subsequent EEG measurements conducted during 1 second (column), the last column represents the label y {1,2,3,4,5} (subjects in class 1 have epileptic seizure, other classes represent different control groups, such as eyes open (5), or eyes closed (4)).

The train and test split and normalization is already done. 
Your task is to classify the given data with k-NN with k=3 (as during previous laboratories) and measure the accuracy.

Next, try to eliminate redundant attributes. Reduce the number of attributes to 30. Use the SelectKBest class. Remember to fit it only on train data. To reduce the test data to obtain only remained parameters use transform method on fit SelectKBest object. Use mutual_info_classif measure to assess the relevance of attributes.


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif

data = pd.read_csv("seizure.csv")
data_Y = data["y"]
data = data.drop(columns=["Name", "y"])

X_train, X_test, y_train, y_test = train_test_split(data, data_Y, test_size=0.33, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#TODO you are given data in X_train, X_test, y_train, y_test variables

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
acc = accuracy_score(y_test, neigh.predict(X_test))
print(acc)

selector = SelectKBest(score_func=mutual_info_classif, k=30)
X_train_reduced = selector.fit_transform(X_train, y_train)
X_test_reduced = selector.transform(X_test)

neigh2 = KNeighborsClassifier(n_neighbors=3)
neigh2.fit(X_train_reduced, y_train)
acc = accuracy_score(y_test, neigh2.predict(X_test_reduced))
print(acc)

In the exercise above we set the number of features arbitrarily. 

**Question** How can we find the optimal number of attributes in machine learning problems?

*moja odpowiedź*


# 2. Clustering

**Question** What is clustering? What can we achieve with clustering?

**Question** What clustering algorithms do you know?

*moja odpowiedź*

# 3. K-means algorithm

**Question** What are the following steps of the k-means algorithm?

**Question** How can we choose the initial clusters?

*moja odpowiedź*

1. Inicjalizacja centroid
2. Przypisanie danych do najbliższej centroidy
3. Przeliczenie nowych centroid (średnia punktów danego klastra)
4. Powtarzać 2-3 aż do określonego warunku końca (np. zmiany mniejszej od zadanego progu, lub ilości iteracji)


Jak wybierać?
- wiedza ekspercka (narzucone z góry dla algorytmu)
- losowo (albo punkt z przestrzeni, albo losowa dana
- wybrać widoczne odstające przypadki (outliers)


**Exercise**
Given the following examples of grades od 5 students we want to divide them into 2 groups:

| Subject | A   | B   |
|---------|-----|-----|
| 1       | 1.0 | 1.0 |
| 2       | 1.5 | 2.0 |
| 3       | 3.0 | 3.0 |
| 4       | 5.0 | 7.0 |
| 5       | 3.5 | 5.0 |

We have chosen the two furthest students (using euclidean distance) as the initial clusters' centroids:

|Cluster|Centroid|A  |B  |
|-------|--------|---|---|
|C1     |k1      |1.0|1.0|
|C2     |k2      |5.0|7.0|

Perform the first iteration of k-means: divide all students into clusters and find the centroids of these clusters.


In [10]:
import math

def dist(p1, p2):
    return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

def mean_point(plist):
    sum_x = sum(point[0] for point in plist)
    sum_y = sum(point[1] for point in plist)
    mean_x = sum_x / len(plist)
    mean_y = sum_y / len(plist)
    return mean_x, mean_y

p1 = (1.0, 1.0)
p2 = (1.5, 2.0)
p3 = (3.0, 3.0)
p4 = (5.0, 7.0)
p5 = (3.5, 5.0)

d21 = dist(p2, p1)
d31 = dist(p3, p1)
d51 = dist(p5, p1)

d24 = dist(p2, p4)
d34 = dist(p3, p4)
d54 = dist(p5, p4)

print(d21, d24)
print(d31, d34)  # punkt 3 należy do klastra C2
print(d51, d54)  # punkt 5 należy do klastra C2

# C1:  S1, S2, S3
# C2: S4, S5

# nowe centroidy
c1 = mean_point([p1, p2, p3])
print(c1) #(1.83, 2.0)
c2 = mean_point([p4, p5])
print(c2) #(4.25, 6.0)


1.118033988749895 6.103277807866851
2.8284271247461903 4.47213595499958
4.716990566028302 2.5
(1.8333333333333333, 2.0)
(4.25, 6.0)


**Question** When the algorithm should stop?

**Question** What advantages and disadvantages of k-means clustering can you find?


*moja odpowiedź*

Jak w opisie wyżej, np. po określonej ilości iteracji, lub gdy zmiany centroid nie przynoszą znaczących efektów (np. centroida przesuwa się o marginalnie małe wartości, lub nie zmieniają się elementy należące do danego klastra)

Zalety:
+ Prostota (a przez to wydajność) rozwiązania
+ Skalowalność (dodanie nawet dużej ilości danych nie jest problemem
+ popularność

Wady:
- Czułość na wartości inicjalizujące (pierwsze centroidy i ilość klastrów)
- założenie kolistych klastrów
- czułość na outlier'y
- problem wielowymiarowych danych (klaster danych zbliżonych na jednym wymiarze nie będzie stworzony z racji na ogromne dystanse na innych wymiarach)

# 4. K-means with scikit-learn

### 4.1. Download files mouse.csv and lines.csv. They have multiple examples described with 2 attributes.  You are given the functions to read files and plot the data. Use these functions to plot data from both files. Can you manually determine 3 clusters in each of the files?

In [ ]:
import csv
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

def read_file(path):
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
        data = [row for row in reader]
        data = StandardScaler().fit_transform(data)
    return np.array(data)

def plot_data(data):
    plt.scatter(data[:,0], data[:, 1])
    plt.show()

In [ ]:
# TODO call functions above and try to find clusters in obtained datasets


### 4.2 Now, let's try to use k-means on the obtained dataset. Again, you are given a function to visualize the obtained plot. Your task is to use KMeans with propoer parameters on "mouse" and "lines" datasets and see if the clusters generated by k-means are the same that you suggested in the previous exercise.

See documentation and examples: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

In [ ]:
def visualize_clusters(clusters, centroids): 
    #clusters: list of numpy arrays (each array with examples in one cluster)
    #centroids: numpy array
    for c in clusters:
        plt.scatter(c[:,0], c[:,1])
    plt.scatter(centroids[:,0], centroids[:,1], marker='+', color='black', s=100)
    plt.show()

In [ ]:
from sklearn.cluster import KMeans
# TODO use KMeans to cluster mouse and lines. Visualize and analyze the obtained clusters.


# 5. Attribute selection in unsupervised problems

As in unsupervised learning problems (such as clustering) we do not have decision classess, so attribute selection methods based on info gain, chi2 etc. are not applicable here.

**Question** What can we do when we want to reduce the number of features in unsupervised problems?

We will try to use two attribute selection methods, which can be used in clustering (unsupervised learning): removing low-variance attributes and removing correlated attributes.

You are given code which reads real-world dataset containing data about customers and saves it in 'data' data frame. 

**Task** Perform attribute selection using the methods above. Firstly, you can use VarianceThreshold class (https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html). Try different values of threashold and compare how many attributes have been removed. Next, you can use corr() method of DataFrame to obtain the correlation matrix. What can we deduce from this matrix? How can we use it to feature selection? Try to implement proper attribute selection based on correlations.

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

data = pd.read_csv("customers.csv")
data = data.drop(columns="Address")
data.head()

In [ ]:
# TODO play with attribute selection 
# ...

# Homework

* Write your own implementation of k-means algorithm with random centroid initialization and 2 stopping conditions: max iterations and centroid convergence (if all attributes of all centroids changes not more than some epsilon the algorithm should stop). 
* Use your implementation to cluster data about cereal products with their dietary characteristics (cereals.csv, 16 attributes). 
* It contains some nominal attributes (name, mfr, type). You can omit the first two of them. Type attribute is binary, so you can replace it with values 0 and 1.
* Perform the clustering of the cereals into 3 groups using k-means algorithm. 
* Remember to preprocess the data: normalization/standardization, attribute selection. 
* Try to describe the obtained groups based on the obtained centroids. What do all cereals within a certain group have in common?
* Write a report containing information about preprocessing methods that you used, number of cereals within each cluster, and your conclusions about the clustering results.

**Deadline +2 weeks**